In [2]:
import numpy as np
from sklearn.model_selection import train_test_split

from keras.layers import Input, Dense
#BatchNormalization, Layer, concatenate
from keras.models import Model
import tensorflow.keras.backend as K
import tensorflow as tf

import energyflow as ef


import matplotlib.pyplot as plt
plt.rc('font', size=20)
plt.rc('font', family = 'serif')

In [3]:
#These are the same datasets from the OmniFold paper https://arxiv.org/abs/1911.09107.  More detail at https://energyflow.network/docs/datasets/.
#Pythia and Herwig are two generators; one will be treated here as the "simulation" and one as "data".
datasets = {'Pythia26': ef.zjets_delphes.load('Pythia26', num_data=1000000),
            'Herwig': ef.zjets_delphes.load('Herwig', num_data=1000000)}

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [4]:
w_true = datasets['Pythia26']['gen_widths']
w_reco = datasets['Pythia26']['sim_widths']
w_true_alt = datasets['Herwig']['gen_widths']
w_reco_alt = datasets['Herwig']['sim_widths']

In [5]:
def weighted_binary_crossentropy(y_true, y_pred):
    weights = tf.gather(y_true, [1], axis=1) # event weights
    y_true = tf.gather(y_true, [0], axis=1) # actual y_true for loss
    
    # Clip the prediction value to prevent NaN's and Inf's
    epsilon = K.epsilon()
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    t_loss = -weights * ((y_true) * K.log(y_pred) +
                         (1 - y_true) * K.log(1 - y_pred))
    return K.mean(t_loss)

In [6]:
xvals_1 = np.concatenate([w_true_alt,w_true])
yvals_1 = np.concatenate([np.ones(len(w_true_alt)),np.zeros(len(w_true))])

In [7]:
#mean

losses_width = []
for lambda1 in np.linspace(-2,10,20):

    weights_1 = np.concatenate([np.ones(len(w_true_alt)),np.exp(lambda1*w_true)*len(w_true_alt)/np.sum(np.exp(lambda1*w_true))])

    X_train_1, X_test_1, Y_train_1, Y_test_1, w_train_1, w_test_1 = train_test_split(xvals_1, yvals_1, weights_1)

    Y_train_2 = np.stack((Y_train_1, w_train_1), axis=1)
    Y_test_2 = np.stack((Y_test_1, w_test_1), axis=1)

    inputs = Input((1, ))
    hidden_layer_1 = Dense(50, activation='relu')(inputs)
    hidden_layer_2 = Dense(50, activation='relu')(hidden_layer_1)
    hidden_layer_3 = Dense(50, activation='relu')(hidden_layer_2)
    outputs = Dense(1, activation='sigmoid')(hidden_layer_3)
    model = Model(inputs=inputs, outputs=outputs)
    
    model.compile(loss=weighted_binary_crossentropy, optimizer='Adam', metrics=['accuracy'])
    model.fit(X_train_1,
              Y_train_2,
              epochs=3,
              batch_size=1000,
              verbose=0)

    losses_width+=[model.history.history['loss'][-1]]
    print(f"{lambda1 = }, loss = {losses_width[-1]}")

2023-12-21 19:59:22.983633: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-21 19:59:25.011983: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 77973 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:03:00.0, compute capability: 8.0
2023-12-21 19:59:25.013644: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 78373 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:41:00.0, compute capability: 8.0
2023-12-21 19:59:25.015128: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/devi

lambda1 = -2.0, loss = 0.6665012240409851
lambda1 = -1.368421052631579, loss = 0.6718755960464478
lambda1 = -0.736842105263158, loss = 0.6770674586296082
lambda1 = -0.10526315789473695, loss = 0.6814423203468323
lambda1 = 0.5263157894736841, loss = 0.6854212284088135
lambda1 = 1.1578947368421053, loss = 0.6882736682891846
lambda1 = 1.789473684210526, loss = 0.6899831891059875
lambda1 = 2.421052631578947, loss = 0.6906856894493103
lambda1 = 3.052631578947368, loss = 0.6902337670326233
lambda1 = 3.6842105263157894, loss = 0.6878787875175476
lambda1 = 4.315789473684211, loss = 0.6840240359306335
lambda1 = 4.947368421052631, loss = 0.6781253814697266
lambda1 = 5.578947368421052, loss = 0.6701961755752563
lambda1 = 6.210526315789473, loss = 0.6605696678161621
lambda1 = 6.842105263157894, loss = 0.6483500599861145
lambda1 = 7.473684210526315, loss = 0.6339715719223022
lambda1 = 8.105263157894736, loss = 0.6176109910011292
lambda1 = 8.736842105263158, loss = 0.5994991064071655
lambda1 = 9.368

In [8]:
#second moment

losses_width2 = []
for lambda2 in np.linspace(-2,10,20):
    
    weights_1 = np.concatenate([np.ones(len(w_true_alt)),np.exp(lambda2*w_true**2)*len(w_true_alt)/np.sum(np.exp(lambda2*w_true**2))])

    X_train_1, X_test_1, Y_train_1, Y_test_1, w_train_1, w_test_1 = train_test_split(xvals_1, yvals_1, weights_1)

    Y_train_2 = np.stack((Y_train_1, w_train_1), axis=1)
    Y_test_2 = np.stack((Y_test_1, w_test_1), axis=1)

    inputs = Input((1, ))
    hidden_layer_1 = Dense(50, activation='relu')(inputs)
    hidden_layer_2 = Dense(50, activation='relu')(hidden_layer_1)
    hidden_layer_3 = Dense(50, activation='relu')(hidden_layer_2)
    outputs = Dense(1, activation='sigmoid')(hidden_layer_3)
    model = Model(inputs=inputs, outputs=outputs)
    
    model.compile(loss=weighted_binary_crossentropy, optimizer='Adam', metrics=['accuracy'])
    model.fit(X_train_1,
              Y_train_2,
              epochs=3,
              batch_size=1000,
              verbose=0)
    losses_width2+=[model.history.history['loss'][-1]]
    print(f"{lambda2 = }, loss = {losses_width2[-1]}")

lambda2 = -2.0, loss = 0.6771501898765564
lambda2 = -1.368421052631579, loss = 0.6787495017051697
lambda2 = -0.736842105263158, loss = 0.6804331541061401
lambda2 = -0.10526315789473695, loss = 0.6818827986717224
lambda2 = 0.5263157894736841, loss = 0.683295726776123
lambda2 = 1.1578947368421053, loss = 0.6845768094062805
lambda2 = 1.789473684210526, loss = 0.6856462955474854
lambda2 = 2.421052631578947, loss = 0.686540424823761
lambda2 = 3.052631578947368, loss = 0.687204897403717
lambda2 = 3.6842105263157894, loss = 0.6875933408737183
lambda2 = 4.315789473684211, loss = 0.6878448128700256
lambda2 = 4.947368421052631, loss = 0.6876720786094666
lambda2 = 5.578947368421052, loss = 0.6873108148574829
lambda2 = 6.210526315789473, loss = 0.6863394379615784
lambda2 = 6.842105263157894, loss = 0.685082197189331
lambda2 = 7.473684210526315, loss = 0.683343231678009
lambda2 = 8.105263157894736, loss = 0.6809446811676025
lambda2 = 8.736842105263158, loss = 0.6781526803970337
lambda2 = 9.36842105

In [13]:
lambda1 = 2.421052631578947
lambda2 = 4.315789473684211
xvals = np.concatenate([w_true_alt,w_true])
yvals = np.concatenate([np.ones(len(w_true_alt)),np.zeros(len(w_true))])
weights = np.concatenate([np.ones(len(w_true_alt)),np.exp(lambda1*w_true + lambda2*w_true**2)*len(w_true_alt)/np.sum(np.exp(lambda1*w_true + lambda2*w_true**2))])



In [12]:
np.savez('npfiles/jetlosses', xvals = xvals, yvals = yvals, weights = weights,
         losses1 = losses_width, losses2 = losses_width2,
         lambda1 = [lambda1], lambda2 = [lambda2])